Requirements

In [ ]:
!pip install pandasql
import pandas as pd
import pandasql

Data Collection (From ChEMBL)

In [3]:
def collect_data(url):
    import requests
    import zipfile
    import io
    response = requests.get(url)
    if response.status_code == 200:
        # Reading zip file from requests response
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))
        # Printing file content in zip
        list_of_files = zip_file.namelist()
        print("ZIP file content:")
        for file_name in list_of_files:
            print(file_name)
        # Extracting file from zip
        zip_file.extractall()
        print("Successfully extracted zip file.")
    else:
        print("Failed to download file. Response:", response.status_code)
    return file_name

In [ ]:
# copy url for csv download here
# TODO: the urls should always be updated before executing
# Add more url to enrich the dataset
# This study utilizes molecules targeting single protein MDM2 (CHEMBL5023) & protein-protein interaction of MDM2-p53 (CHEMBL1907611)
urls = [
    "https://www.ebi.ac.uk/chembl/interface_api/delayed_jobs/outputs/DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=/DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=.zip",
    "https://www.ebi.ac.uk/chembl/interface_api/delayed_jobs/outputs/DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=/DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=.zip"
]

In [ ]:
file_names = []
for url in urls:
    # store file name in file_names
    file_names.append(collect_data(url))

In [3]:
# In case you don't want to redownload the csv files online
# run this block to read csv file names available in current directory
# if you ran previous block above, just skip this block
from pathlib import Path
file_names = []
files = Path('./')
for file in files.iterdir():
    if 'DOWNLOAD' in file.name:
        file_names.append(file.name)

Standard Cleaning Only

In [4]:
def clean_data(files):
    import numpy as np
    # initiate data_main, main data frame that will be used to aggregate all csv
    # only use chembl ID, smile, and standard value columns
    data_main = pd.DataFrame({'Molecule ChEMBL ID':[], 'Smiles':[], 'Standard Value':[]})
    # for every csv downloaded, do these following tasks
    for file in files:
        # read data
        data = pd.read_csv(file, sep=';')
        # drop rows if Smiles or Standard Value is null
        removed_null = data.dropna(subset=['Smiles', 'Standard Value'])
        # slice data from initial data frame. only use columns in data_main
        sliced = removed_null[data_main.columns]
        # convert IC50 to pIC50
        # convert IC50 from nM to M
        sliced.loc[:, ['Standard Value']] = sliced[['Standard Value']].apply(lambda x: x*10**-9, axis=1)
        # transform to pIC50
        sliced.loc[:, ['Standard Value']] = -np.log10(sliced[['Standard Value']])
        # join data
        data_main = pd.concat([data_main, sliced])
    # drop duplicates according to Smiles column
    data_main = data_main.drop_duplicates(subset=['Smiles'])
    return data_main

In [5]:
# store result of standard-cleaned dataset in df_main
df_main = clean_data(file_names)

In [6]:
df_main

,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768
...,...,...,...
1611,CHEMBL5191434,CCC[C@H]1N(C(=O)c2cnccc2C)CCC[C@@]1(Oc1csc(C(F...,7.468521
1614,CHEMBL5191746,CCC[C@H]1N(C(=O)c2cnccc2C(F)(F)F)CCC[C@@]1(Oc1...,7.522879
1615,CHEMBL5179820,CCC[C@H]1N(C(=O)c2c(C(F)(F)F)ccnc2O)CCC[C@@]1(...,7.958607
1616,CHEMBL5207199,CCC[C@H]1N(C(=O)c2c(C(F)(F)F)ccnc2OCC(=O)O)CCC...,7.602060


In [7]:
df_main[['Smiles', 'Standard Value']].to_csv('D1_std.csv', index=False)

Standard Cleaning + Filtering

In [8]:
def filter_data(files):
    import numpy as np
    # initiate data_main, main data frame that will be used to aggregate all csv
    # only use chembl ID, smile, and standard value columns
    data_main = pd.DataFrame({'Assay ChEMBL ID':[], 'Molecule ChEMBL ID':[], 'Smiles':[], 'Standard Relation':[], 'Standard Value':[], 'Molecular Weight':[]})
    # for every csv downloaded, do these following tasks
    for file in files:
        # read data
        data = pd.read_csv(file, sep=';')
        # drop rows if Smiles or Standard Value is null
        removed_null = data.dropna(subset=['Smiles', 'Standard Value', 'Standard Relation'])
        # slice data from initial data frame. only use columns in data_main
        sliced = removed_null[data_main.columns]
        # filter the standard relation to only "="
        sliced = sliced[sliced['Standard Relation'].str.contains('=')]
        # filter 100 < MW < 900 (Small molecules)
        sliced = sliced[(sliced['Molecular Weight'] > 100) & (sliced['Molecular Weight'] < 750)]
        # convert IC50 to pIC50
        # convert IC50 from nM to M
        sliced.loc[:, ['Standard Value']] = sliced[['Standard Value']].apply(lambda x: x*10**-9, axis=1)
        # transform to pIC50
        sliced.loc[:, ['Standard Value']] = -np.log10(sliced[['Standard Value']])
        # join data
        data_main = pd.concat([data_main, sliced])
    # drop duplicates according to Smiles column
    data_main = data_main.drop_duplicates(subset=['Smiles'])
    # data_main = data_main.drop_duplicates(subset=['Smiles'])
    return data_main[['Assay ChEMBL ID', 'Molecule ChEMBL ID', 'Smiles', 'Standard Value']]

In [9]:
# store result of filtered data in df_main_2
df_main_2 = filter_data(file_names)

In [10]:
df_main_2

,Assay ChEMBL ID,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3370341,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3370341,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3370341,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3370341,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3370341,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768
...,...,...,...,...
1609,CHEMBL4681440,CHEMBL4744488,CC(=O)N[C@@H]1[C@H](C)N(CC2CC2)[C@@]2(C(=O)Nc3...,6.640165
1610,CHEMBL5110382,CHEMBL5180348,CCOC(=O)C1=NN(c2ccc(Cl)cc2)C2(C(=O)Nc3cc(Cl)cc...,7.150581
1611,CHEMBL5135142,CHEMBL5191434,CCC[C@H]1N(C(=O)c2cnccc2C)CCC[C@@]1(Oc1csc(C(F...,7.468521
1614,CHEMBL5135142,CHEMBL5191746,CCC[C@H]1N(C(=O)c2cnccc2C(F)(F)F)CCC[C@@]1(Oc1...,7.522879


In [11]:
df_main_2[['Smiles', 'Standard Value']].to_csv('D1_filtered.csv', index=False)

Standard Cleaning + Filtering + Selective Cleaning

In [12]:
def filter_data_2(files):
    import numpy as np
    # initiate data_main, main data frame that will be used to aggregate all csv
    # only use chembl ID, smile, and standard value columns
    data_main = pd.DataFrame({'Assay ChEMBL ID':[], 'Molecule ChEMBL ID':[], 'Smiles':[], 'Standard Relation':[], 'Standard Value':[], 'Molecular Weight':[]})
    # for every csv downloaded, do these following tasks
    for file in files:
        # read data
        data = pd.read_csv(file, sep=';')
        # drop rows if Smiles or Standard Value is null
        removed_null = data.dropna(subset=['Smiles', 'Standard Value', 'Standard Relation'])
        # slice data from initial data frame. only use columns in data_main
        sliced = removed_null[data_main.columns]
        # filter the standard relation to only "="
        sliced = sliced[sliced['Standard Relation'].str.contains('=')]
        # filter 100 < MW < 900 (Small molecules)
        sliced = sliced[(sliced['Molecular Weight'] > 100) & (sliced['Molecular Weight'] < 750)]
        # convert IC50 to pIC50
        # convert IC50 from nM to M
        sliced.loc[:, ['Standard Value']] = sliced[['Standard Value']].apply(lambda x: x*10**-9, axis=1)
        # transform to pIC50
        sliced.loc[:, ['Standard Value']] = -np.log10(sliced[['Standard Value']])
        # join data
        data_main = pd.concat([data_main, sliced])
    # data_main = data_main.drop_duplicates(subset=['Smiles'])
    return data_main[['Assay ChEMBL ID', 'Molecule ChEMBL ID', 'Smiles', 'Standard Value']]

In [13]:
# store result of filtered data for selective cleaning in df_main_3
df_main_3 = filter_data_2(file_names)

In [14]:
df_main_3

,Assay ChEMBL ID,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3370341,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3370341,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3370341,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3370341,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3370341,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768
...,...,...,...,...
1614,CHEMBL5135142,CHEMBL5191746,CCC[C@H]1N(C(=O)c2cnccc2C(F)(F)F)CCC[C@@]1(Oc1...,7.522879
1617,CHEMBL5030008,CHEMBL5079411,C[C@@H](Nc1nc(C(=O)O)nc2nc(NC3CCCCC3)n(Cc3ccc(...,8.119186
1618,CHEMBL5030008,CHEMBL5090666,C[C@H]1CC[C@H](Cn2c(N3CCOC[C@H]3c3ccccc3)nc3nc...,8.000000
1619,CHEMBL5030008,CHEMBL3647064,C[C@@H]1COC[C@@H](C)N1c1nc2cc(-c3noc(=O)[nH]3)...,8.744727


Grouping & Sorting (2 Layers)

In [15]:
def sorting_data(data):
    # import library to handle sql query (not suitable to pandas)
    from pandasql import sqldf
    # transform to list > connected with '_' (common separator style in sql)
    df_cols = ['_'.join(col.split(' ')) for col in data.columns]
    # copy the data frame to df_mod
    df_mod = data.copy()
    # change column name
    df_mod.columns = df_cols
    query = """
    SELECT
    Assay_ChEMBL_ID,
    Smiles,
    MAX(Standard_Value) AS pIC50
    FROM df_mod
    -- Agregate based on the assay procedure and smiles respectively, in each assay > look for the max value
    GROUP BY Assay_ChEMBL_ID, Smiles
    -- Sort by the size of assay procedure
    ORDER BY COUNT(Assay_ChEMBL_ID) DESC;
    """
    df_sorted = sqldf(query)
    return df_sorted

In [16]:
df_sort = sorting_data(df_main_3)

Executing Selective Cleaning

In [17]:
def selective_removal(data):
    print(f'Before dropping duplicates: {data.shape}')
    data = data.drop_duplicates(subset=['Smiles'])
    print(f'After dropping duplicates: {data.shape}')
    return data

In [18]:
df_sc = selective_removal(df_sort)

Before dropping duplicates: (3726, 3)
After dropping duplicates: (2954, 3)


In [19]:
df_sc

,Assay_ChEMBL_ID,Smiles,pIC50
0,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCCC4CCCC43)nc3cc(-c4noc...,9.000000
2,CHEMBL3706024,COCC(C)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cncc(Cl)c...,9.000000
3,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCO[C@H]4CC(F)C[C@@H]43)...,9.000000
5,CHEMBL3706331,C[C@H]1CC[C@H](Cn2c(N3CCO[C@@H]4CC(F)C[C@H]43)...,9.000000
6,CHEMBL1838865,N#Cc1ccc(CN2C(=O)c3cccc(Cl)c3C2(OCC2(CO)CC2)c2...,6.866461
...,...,...,...
3720,CHEMBL965739,COc1ccc(C2=NC(c3ccc(Cl)cc3)C(c3ccc(Cl)cc3)N2)c...,7.045757
3722,CHEMBL988050,COc1c(C)c(CCN(C)C)c2c(c1OC)SSSSS2,3.917933
3723,CHEMBL988050,COc1cc(CCN(C)C)c2c(c1O)SSSSS2,4.232102
3724,CHEMBL988050,CSC1=C(CCN)c2nc3ccccc3c3ccnc(c23)C1=O,3.994391


In [20]:
df_sc[['Smiles', 'pIC50']].to_csv('D1_full_opt.csv', index=False)